In [80]:
import sklearn 
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima_model import ARMA
import datetime
import re

pd.options.display.max_rows = 999
pd.set_option('display.max_columns', 500)

# Load CSVs

In [42]:
regular_season_results = pd.read_csv("../ncaa_data/RegularSeasonDetailedResults.csv")
tourney_results = pd.read_csv("../ncaa_data/NCAATourneyDetailedResults.csv")
secondary_results = pd.read_csv("../ncaa_data/SecondaryTourney")

results = pd.concat([regular_season_results, tourney_results])

In [43]:
seasons = pd.read_csv("../ncaa_data/Seasons.csv")
seasons = seasons[["Season","DayZero"]]
results = results.set_index("Season").join(seasons.set_index("Season"), how="inner", rsuffix="season").reset_index()

In [44]:
results["WFGM_Perc"] = results["WFGM"]/results["WFGA"]
results["LFGM_Perc"] = results["LFGM"]/results["LFGA"]
results["WFGM3_Perc"] = results["WFGM3"]/results["WFGA3"]
results["LFGM3_Perc"] = results["LFGM3"]/results["LFGA3"]
results["WFTM_Perc"] = results["WFTM"]/results["WFTA"]
results["LFTM_Perc"] = results["LFTM"]/results["LFTA"]

In [ ]:
player_df = pd.read_csv("../ncaa_data/Players_2010.csv")

# Build Team/Season Features

In [112]:
winning_results = results[['Season', 'DayNum','DayZero', 'WTeamID', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',"WFGM_Perc","WFGM3_Perc","WFTM_Perc"]]
losing_results = results[['Season', 'DayNum','DayZero', 'LTeamID', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',"LFGM_Perc","LFGM3_Perc","LFTM_Perc"]]
winning_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score','FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF',"FGM_Perc","FGM3_Perc","FTM_Perc"]
losing_results.columns = ['Season', 'DayNum','DayZero', 'TeamID', 'Score', 'FGM', 'FGA', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR',
'Ast', 'TO', 'Stl', 'Blk', 'PF',"WFGM_Perc","FGM3_Perc","FTM_Perc"]
team_df = pd.concat([winning_results, losing_results])


In [113]:
seeds = pd.read_csv("../ncaa_data/NCAATourneySeeds.csv")
seeds = seeds.set_index(["Season","TeamID"])
team_df = team_df.reset_index().join(seeds, how="inner", on=["Season","TeamID"])

In [114]:
team_df["Seed"] = team_df["Seed"].apply(lambda x:int(re.search(r'\d+', x).group()))
team_df["Seed"] = team_df["Seed"].fillna(20)

# Bring in Massey Ordinals 

In [115]:
massey = pd.read_csv("../ncaa_data/MasseyOrdinals.csv")
massey = massey.rename(columns={"RankingDayNum":"DayNum"})
massey = massey.pivot_table(index=["Season","TeamID","DayNum"], columns="SystemName", values="OrdinalRank")
massey = massey.fillna(method="ffill")
massey = massey.drop("DC2", axis=1)
massey = massey.fillna(massey.mean())

In [116]:
team_df = team_df.set_index(["Season","TeamID","DayNum"]).join(massey, how="left")


In [123]:
team_df = team_df.reset_index().sort_values(["Season","DayNum"]).set_index("DayNum").groupby(["Season","TeamID"]).apply(lambda x: x.reindex(range(0,150), method='ffill').ffill())

# Clean Up and Make Some Columns MA

In [124]:
team_df = team_df.drop(["TeamID","Season"], axis=1)
deltas = team_df.reset_index()["DayNum"].apply(lambda x: datetime.timedelta(days=x))
team_df["date"] = pd.to_datetime(team_df["DayZero"]) + deltas.values

In [125]:
exclude = ["date","DayZero","Score","DayNum","Season"]
ma_columns = team_df.columns.tolist()
ma_columns = [item for item in ma_columns if item not in exclude]

In [126]:
ma_data = team_df.groupby(["Season","TeamID"])[ma_columns].apply(lambda x: x.rolling(window=20, min_periods=1).mean())

In [127]:
team_df[ma_columns] = ma_data

# Pickle Team DF

In [128]:
team_df.to_pickle("../ncaa_data/team_df.p")

# Build Game DF

In [129]:
games = results[["WTeamID","LTeamID","Season","DayNum","WLoc","DayZero","WScore","LScore"]]

In [130]:
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)


/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [131]:
games["Team1Score"] = 0
games["Team2Score"] = 0
games["Team1"] = games[["WTeamID","LTeamID"]].min(axis=1)
games["Team2"] = games[["WTeamID","LTeamID"]].max(axis=1)
games.loc[games["WTeamID"] == games["Team1"],"Team1Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team1"],"Team1Score"] = games[games["WTeamID"] != games["Team1"]]["LScore"]
games.loc[games["WTeamID"] == games["Team2"],"Team2Score"] = games["WScore"]
games.loc[games["WTeamID"] != games["Team2"],"Team2Score"] = games["LScore"]
# games["ScoreDiff"] = 
games["Team1Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team1"])
games["Team2Home"] = (games["WLoc"]=="H") & (games["WTeamID"] == games["Team2"])

/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/cody/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [132]:
games = games.drop(["WTeamID","LTeamID","WLoc","DayZero"], axis=1)


# Increment the dayNum so that we only use results up through the day previous to the game

In [133]:
incremented = team_df.reset_index()
incremented["DayNum"] = incremented["DayNum"] + 1
incremented = incremented.set_index(["Season","TeamID","DayNum"])
full_df = games.join(incremented, on=["Season", "Team1","DayNum"], how="right").join(incremented, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

# Add Some Diff Columns

In [134]:
for column in ma_columns:
    try:
        full_df[column + "diff"] = full_df[column] - full_df[str(column +"2")]
    except Exception as e:
        print(e)

In [135]:
full_df.to_pickle("../ncaa_data/full_df.p")

In [136]:
full_df.tail()

,Season,DayNum,WScore,LScore,Team1,Team2,Team1Score,Team2Score,Team1Home,Team2Home,index,Ast,Blk,DR,DayZero,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Score,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INP,ISR,JCI,JEN,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGY,MIC,MKV,MOR,MPI,MSX,MUZ,MvG,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WMR,WOB,WOL,WTE,YAG,ZAM,date,index2,Ast2,Blk2,DR2,DayZero2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Score2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,CNG2,COL2,CPA2,CPR2,CRO2,CRW2,CTL2,D1A2,DAV2,DC2,DCI2,DDB2,DES2,DII2,DOK2,DOL2,DUN2,...,MPI2,MSX2,MUZ2,MvG2,NOL2,NOR2,OCT2,OMY2,PEQ2,PGH2,PH2,PIG2,PKL2,PMC2,POM2,PPR2,PRR2,PTS2,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WMR2,WOB2,WOL2,WTE2,YAG2,ZAM2,date2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGYdiff,MICdiff,MKVdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WMRdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
77609,2017,144,70.0,50.0,1124,1376,50.0,70.0,False,False,77434.50,15.95,2.50,22.15,10/31/2016,58.35,17.35,28.0,7.2,0.419311,0.476926,15.35,10.75,0.688192,15.65,17.65,82.0,5.90,13.15,0.459762,3.0,20.0,274.0,168.0,6.0,4.0,242.0,7.0,284.0,313.0,3.0,226.0,299.0,115.0,278.0,279.0,228.0,192.0,291.0,8.0,12.0,198.0,330.0,13.0,2.0,308.0,285.0,12.0,154.0,292.0,12.0,11.0,1.0,15.0,5.0,13.0,9.0,12.0,4.0,15.0,174.0,121.0,16.0,238.0,315.0,265.0,3.0,4.0,8.0,16.0,248.0,259.0,187.0,13.0,209.0,320.0,146.0,12.0,266.0,255.0,273.0,13.0,268.0,170.0,200.0,91.0,253.0,4.0,275.0,282.0,233.0,282.0,165.0,2.0,9.0,3.0,12.0,14.0,222.0,202.0,4.0,177.0,15.0,254.0,237.0,232.0,18.0,255.0,276.0,12.0,292.0,9.0,264.0,174.0,191.0,253.0,3.0,271.0,21.0,198.0,170.0,9.0,186.0,9.0,170.0,286.0,197.0,170.0,3.0,308.0,268.0,183.0,181.0,1.0,243.0,12.0,13.0,214.0,3.0,5.0,276.0,13.0,189.0,247.0,180.0,288.0,3.0,16.0,142.0,230.0,2.0,9.0,14.0,5.0,2.0,284.0,8.0,222.0,283.0,175.0,297.0,298.0,9.0,11.0,15.0,28.0,209.0,201.0,253.0,193.0,8.0,7.0,10.0,244.0,3.0,4.0,271.0,170.0,182.0,2017-03-23,77449.95,12

# Match Ups to Predict

In [137]:
pairs_to_predict = pd.read_csv("../ncaa_data/SampleSubmissionStage1.csv")["ID"].str.split("_")
seasons = pairs_to_predict.apply(lambda x: int(x[0]))
team1 = pairs_to_predict.apply(lambda x: int(x[1]))
team2 = pairs_to_predict.apply(lambda x:int(x[2]))

In [138]:
tourney_matchups = pd.DataFrame()
tourney_matchups["Team1"] = team1
tourney_matchups["Team2"] = team2
tourney_matchups["Season"] = seasons
tourney_matchups["DayNum"] = 149
tourney_matchups["Team1Home"] = 0
tourney_matchups["Team2Home"] = 0

In [139]:
last_games = team_df.reset_index()[team_df.reset_index()["DayNum"]==149]

In [140]:
last_games = last_games.set_index(["Season","TeamID","DayNum"])

In [141]:
tourney_matchups = tourney_matchups.join(last_games, on=["Season", "Team1","DayNum"], how="right").join(last_games, on=["Season", "Team2","DayNum"], rsuffix="2", how="inner")

In [142]:
for column in ma_columns:
    try:
        tourney_matchups[column + "diff"] = tourney_matchups[column] - tourney_matchups[str(column +"2")]
    except Exception as e:
        print(e)

In [143]:
tourney_matchups.to_pickle("../ncaa_data/tourney_matchups.p")

In [145]:
full_df.tail()

,Season,DayNum,WScore,LScore,Team1,Team2,Team1Score,Team2Score,Team1Home,Team2Home,index,Ast,Blk,DR,DayZero,FGA,FGA3,FGM,FGM3,FGM3_Perc,FGM_Perc,FTA,FTM,FTM_Perc,OR,PF,Score,Stl,TO,WFGM_Perc,Seed,7OT,ACU,ADE,AP,ARG,AUS,BBT,BCM,BD,BIH,BKM,BLS,BNM,BOB,BOW,BP5,BPI,BRZ,BUR,BWE,CJB,CMV,CNG,COL,CPA,CPR,CRO,CRW,CTL,D1A,DAV,DC,DCI,DDB,DES,DII,DOK,DOL,DUN,DWH,EBB,EBP,ECK,ENT,ERD,ESR,FAS,FMG,FSH,GC,GRN,GRS,HAS,HAT,HER,HKB,HKS,HOL,HRN,IMS,INP,ISR,JCI,JEN,JNG,JON,JRT,KBM,KEL,KLK,KMV,KOS,KPI,KPK,KRA,LMC,LOG,LYD,LYN,MAS,MB,MCL,MGY,MIC,MKV,MOR,MPI,MSX,MUZ,MvG,NOL,NOR,OCT,OMY,PEQ,PGH,PH,PIG,PKL,PMC,POM,PPR,PRR,PTS,RAG,REI,REN,REW,RIS,RM,ROG,ROH,RPI,RSE,RSL,RT,RTB,RTH,RTP,RTR,SAG,SAP,SAU,SCR,SE,SEL,SFX,SGR,SIM,SMN,SMS,SP,SPR,SPW,STF,STH,STR,STS,TBD,TMR,TOL,TPR,TRK,TRP,TRX,TSR,TW,UCS,UPS,USA,WIL,WLK,WMR,WOB,WOL,WTE,YAG,ZAM,date,index2,Ast2,Blk2,DR2,DayZero2,FGA2,FGA32,FGM2,FGM32,FGM3_Perc2,FGM_Perc2,FTA2,FTM2,FTM_Perc2,OR2,PF2,Score2,Stl2,TO2,WFGM_Perc2,Seed2,7OT2,ACU2,ADE2,AP2,ARG2,AUS2,BBT2,BCM2,BD2,BIH2,BKM2,BLS2,BNM2,BOB2,BOW2,BP52,BPI2,BRZ2,BUR2,BWE2,CJB2,CMV2,CNG2,COL2,CPA2,CPR2,CRO2,CRW2,CTL2,D1A2,DAV2,DC2,DCI2,DDB2,DES2,DII2,DOK2,DOL2,DUN2,...,MPI2,MSX2,MUZ2,MvG2,NOL2,NOR2,OCT2,OMY2,PEQ2,PGH2,PH2,PIG2,PKL2,PMC2,POM2,PPR2,PRR2,PTS2,RAG2,REI2,REN2,REW2,RIS2,RM2,ROG2,ROH2,RPI2,RSE2,RSL2,RT2,RTB2,RTH2,RTP2,RTR2,SAG2,SAP2,SAU2,SCR2,SE2,SEL2,SFX2,SGR2,SIM2,SMN2,SMS2,SP2,SPR2,SPW2,STF2,STH2,STR2,STS2,TBD2,TMR2,TOL2,TPR2,TRK2,TRP2,TRX2,TSR2,TW2,UCS2,UPS2,USA2,WIL2,WLK2,WMR2,WOB2,WOL2,WTE2,YAG2,ZAM2,date2,indexdiff,Astdiff,Blkdiff,DRdiff,FGAdiff,FGA3diff,FGMdiff,FGM3diff,FGM3_Percdiff,FGM_Percdiff,FTAdiff,FTMdiff,FTM_Percdiff,ORdiff,PFdiff,Stldiff,TOdiff,WFGM_Percdiff,Seeddiff,7OTdiff,ACUdiff,ADEdiff,APdiff,ARGdiff,AUSdiff,BBTdiff,BCMdiff,BDdiff,BIHdiff,BKMdiff,BLSdiff,BNMdiff,BOBdiff,BOWdiff,BP5diff,BPIdiff,BRZdiff,BURdiff,BWEdiff,CJBdiff,CMVdiff,CNGdiff,COLdiff,CPAdiff,CPRdiff,CROdiff,CRWdiff,CTLdiff,D1Adiff,DAVdiff,DCdiff,DCIdiff,DDBdiff,DESdiff,DIIdiff,DOKdiff,DOLdiff,DUNdiff,DWHdiff,EBBdiff,EBPdiff,ECKdiff,ENTdiff,ERDdiff,ESRdiff,FASdiff,FMGdiff,FSHdiff,GCdiff,GRNdiff,GRSdiff,HASdiff,HATdiff,HERdiff,HKBdiff,HKSdiff,HOLdiff,HRNdiff,IMSdiff,INPdiff,ISRdiff,JCIdiff,JENdiff,JNGdiff,JONdiff,JRTdiff,KBMdiff,KELdiff,KLKdiff,KMVdiff,KOSdiff,KPIdiff,KPKdiff,KRAdiff,LMCdiff,LOGdiff,LYDdiff,LYNdiff,MASdiff,MBdiff,MCLdiff,MGYdiff,MICdiff,MKVdiff,MORdiff,MPIdiff,MSXdiff,MUZdiff,MvGdiff,NOLdiff,NORdiff,OCTdiff,OMYdiff,PEQdiff,PGHdiff,PHdiff,PIGdiff,PKLdiff,PMCdiff,POMdiff,PPRdiff,PRRdiff,PTSdiff,RAGdiff,REIdiff,RENdiff,REWdiff,RISdiff,RMdiff,ROGdiff,ROHdiff,RPIdiff,RSEdiff,RSLdiff,RTdiff,RTBdiff,RTHdiff,RTPdiff,RTRdiff,SAGdiff,SAPdiff,SAUdiff,SCRdiff,SEdiff,SELdiff,SFXdiff,SGRdiff,SIMdiff,SMNdiff,SMSdiff,SPdiff,SPRdiff,SPWdiff,STFdiff,STHdiff,STRdiff,STSdiff,TBDdiff,TMRdiff,TOLdiff,TPRdiff,TRKdiff,TRPdiff,TRXdiff,TSRdiff,TWdiff,UCSdiff,UPSdiff,USAdiff,WILdiff,WLKdiff,WMRdiff,WOBdiff,WOLdiff,WTEdiff,YAGdiff,ZAMdiff
77609,2017,144,70.0,50.0,1124,1376,50.0,70.0,False,False,77434.50,15.95,2.50,22.15,10/31/2016,58.35,17.35,28.0,7.2,0.419311,0.476926,15.35,10.75,0.688192,15.65,17.65,82.0,5.90,13.15,0.459762,3.0,20.0,274.0,168.0,6.0,4.0,242.0,7.0,284.0,313.0,3.0,226.0,299.0,115.0,278.0,279.0,228.0,192.0,291.0,8.0,12.0,198.0,330.0,13.0,2.0,308.0,285.0,12.0,154.0,292.0,12.0,11.0,1.0,15.0,5.0,13.0,9.0,12.0,4.0,15.0,174.0,121.0,16.0,238.0,315.0,265.0,3.0,4.0,8.0,16.0,248.0,259.0,187.0,13.0,209.0,320.0,146.0,12.0,266.0,255.0,273.0,13.0,268.0,170.0,200.0,91.0,253.0,4.0,275.0,282.0,233.0,282.0,165.0,2.0,9.0,3.0,12.0,14.0,222.0,202.0,4.0,177.0,15.0,254.0,237.0,232.0,18.0,255.0,276.0,12.0,292.0,9.0,264.0,174.0,191.0,253.0,3.0,271.0,21.0,198.0,170.0,9.0,186.0,9.0,170.0,286.0,197.0,170.0,3.0,308.0,268.0,183.0,181.0,1.0,243.0,12.0,13.0,214.0,3.0,5.0,276.0,13.0,189.0,247.0,180.0,288.0,3.0,16.0,142.0,230.0,2.0,9.0,14.0,5.0,2.0,284.0,8.0,222.0,283.0,175.0,297.0,298.0,9.0,11.0,15.0,28.0,209.0,201.0,253.0,193.0,8.0,7.0,10.0,244.0,3.0,4.0,271.0,170.0,182.0,2017-03-23,77449.95,12